# Frequency Analysis, Part 2

## Your name goes here. \(Double\-click on this cell to edit it.\)

Remember to hit Ctrl-Enter after editing any cell.



## Commands in python introduced in this module

---
>
___

## Math topics used in this module

* affine ciphers
* mods
* inverses mod $n$

## Objectives

In this activity we will use Python to conduct a frequency analysis of ciphertext. We'll see how a frequency analysis can help us decrypt shift ciphers and affine ciphers.

## Preliminaries

In this section, we'll import any libraries we need and load the functions we've developed in past modules. Actually, we can tidy things up a little by noting that shift encryption is just a special case of affine encryption where $m = 1$. (I've changed the code below so that $m = 1$ is the default value of $m$. If we want shift encryption, that allows us to leave out the argument for $m$ altogether if we want.) So we really only have to load functions for affine encryption/decryption and frequency analysis.

To make it easier to run, I've put all the code in one big code cell. That's not a generally recommended practice, but it means you can just click inside the cell below and run it using Ctrl-Enter. (Throughout the module, you will need to remember to click on code cells and hit Ctrl-Enter.)

In [ ]:
##### Import libraries #####

import string
import math
import matplotlib.pyplot as plt


##### Helper functions #####

def convert_letter_to_number(letter):
    # Define constant that will allow us to move the letter codes by 97
    MOVE_NUMBER = 97
    # Convert input to ASCII (97 to 122)
    number_ascii = ord(letter)
    # Move number to lie in the range 0 to 25
    number = number_ascii - MOVE_NUMBER
    return number

def convert_number_to_letter(number):
    # Define constant that will allow us to move the letter codes by 97
    MOVE_NUMBER = 97
    # Move number to lie in the ASCII range 97 to 122
    number_ascii = number + MOVE_NUMBER
    # Convert number to letter
    letter = chr(number_ascii)
    return letter

def generate_units(n):
    unit_list = []
    for m in range(1, n):
        if math.gcd(m, n) == 1:
            unit_list.append(m)
    return unit_list


##### Affine encryption functions #####

def affine_encrypt_letter(plaintext_letter, key, m = 1):
    # Replace uppercase with lowercase
    plaintext_letter = plaintext_letter.lower()
    # Check if plaintext_letter is a lowercase letter
    if ord(plaintext_letter) >= 97 and ord(plaintext_letter) <= 122:
        plaintext_number = convert_letter_to_number(plaintext_letter)
        ciphertext_number = (m* plaintext_number + key) % 26
        ciphertext_letter = convert_number_to_letter(ciphertext_number)
        return ciphertext_letter.upper()
    else:
        # If we get to this spot, we know we don't have a letter at all,
        # so just return the character as is
        return plaintext_letter

def affine_encrypt(plaintext, key, m = 1):
    # Stop if m is not chosen appropriately
    if math.gcd(m, 26) != 1:
        print("The value of m must be relatively prime to 26!")
    else:
        ciphertext = ""
        for letter in plaintext:
            ciphertext_letter = affine_encrypt_letter(letter, key, m)
            ciphertext = ciphertext + ciphertext_letter
        return ciphertext

def affine_decrypt(ciphertext, key, m = 1):
    # Stop if m is not chosen appropriately
    if math.gcd(m, 26) != 1:
        print("The value of m must be relatively prime to 26!")
    else:
        m_inv = pow(m, -1, mod = 26)
        plaintext = affine_encrypt(ciphertext, -m_inv*key, m_inv)
        return plaintext.lower()


##### Frequency analysis functions #####

def freq_bar(letter_frequencies):
    plt.bar(*zip(*letter_frequencies.items()))
    plt.show()

def analyze_frequencies(text, output_format = "count", sort = False):
    # Store uppercase alphabet
    alph_upper = string.ascii_uppercase
    # Convert text to uppercase
    text = text.upper()
    # Initialize frequencies to 0
    frequencies = {}
    for letter in alph_upper:
        frequencies[letter] = 0
    # Go through the text and for each letter,
    # add 1 to the value corresponding to that letter
    for letter in text:
        if letter in alph_upper:
            frequencies[letter] += 1
    # Check output_format argument
    if output_format not in ["count", "percent"]:
        print('The output format must be either "count" or "percent".')
        return
    elif output_format == "percent":
        total_letters = sum(frequencies.values())
        for letter in frequencies:
            frequencies[letter] = round(100 * frequencies[letter]/total_letters, ndigits = 3)
    # Check sort argument
    if sort:
        frequencies = dict(sorted(frequencies.items(), key = lambda item: item[1], reverse = True))
    print(frequencies)
    # Create bar graph
    freq_bar(frequencies)


##### English letter frequencies (alphebtic and sorted by frequency) #####

ENGLISH_LETTER_FREQ = {
    "A": 8.167, "B": 1.492, "C": 2.782, "D": 4.253, "E": 12.702, "F": 2.228, "G": 2.015, "H": 6.094, "I": 6.966, "J": 0.153, "K": 0.772, "L": 4.025, "M": 2.406, "N": 6.749, "O": 7.507, "P": 1.929, "Q": 0.095, "R": 5.987, "S": 6.327, "T": 9.056, "U": 2.758, "V": 0.978, "W": 2.360, "X": 0.150, "Y": 1.974, "Z": 0.074
}

ENGLISH_LETTER_FREQ_SORT = dict(sorted(ENGLISH_LETTER_FREQ.items(), key = lambda item: item[1], reverse = True))

## Frequency analysis of shift ciphers

Recall that with a shift cipher, once we know a single letter, we have the key. Therefore, we need to write a function that will allow us to guess a letter (based on a frequency analysis) and use that guess to try to decrypt the ciphertext. Here is the ciphertext we we use:

In [ ]:
ciphertext_example1 = "KYZJKVOKZJRCZKKCVLELJLRCZEKYRKKZJRTKLRCCPKYVDFJKTFDDFECVKKVI"

Without spacing or punctuation, we have very few hints here.

Of course, with shift ciphers, there are only 25 possible keys, so we could brute force this ciphertext and find the message by inspection. Nevertheless, we'll use frequency analysis to show how it works. In future examples where brute force is difficult or impossible, frequency analysis will be the only tool at our disposal.

### Exercise 1a

What is the most common letter in the English language? You probably know the answer to that question off the top of your head. What is the second-most common letter? You may be a little more uncertain about that one.

Using the dictionary `ENGLISH_LETTER_FREQ_SORT`, make a bar chart of the frequencies of English letters. Use the `freq_bar` function as defined above. Then write up your answer to the questions above.

In [ ]:
# Add code here to make a bar chart

<span style='color:#9c27b0'>_Please write up your answer here._</span>


*****

The first thing to do when analyzing a ciphertext is to conduct a frequency analysis. We will use the `analyze_frequencies` function:

In [ ]:
analyze_frequencies(ciphertext_example1, output_format = "percent", sort = True)

The letter "K" is the most frequent letter by far. We will guess that this is the plaintext letter "e".

### Exercise 1b

Use the `analyze_frequencies` function to analyze the text on page 99 of your course pack. You can copy paste the encrypted message into the first code chunk, then use the function to analyze the code chunk.

In [1]:
# add the text here in between the ''' ''' to store the encrypted message so we can analyze it. 
coursepack_encryption= ''' '''

In [ ]:
# Add code here to analyze frequencies of pg 99 text

### Exercise 2

Is the ciphertext letter "K" guaranteed to correspond to the plaintext letter "e"? What kinds of ciphertexts would give you more confidence in that assertion?

<span style='color:#9c27b0'>_Please write up your answer here._</span>


*****

Here is a basic shell for the function we will develop:

In [ ]:
def shift_decrypt_freq_test(ciphertext, ciphertext_letter_guess, plaintext_letter_guess = "e"):
    # Do stuff here
    return(plaintext)

### Exercise 3

In your own words, describe the steps you would use to implement the algorithm. You don't need to write any code. Just explain what the algorithm will need to do with the inputs `ciphertext_letter_guess` and `plaintext_letter_guess` and how that will lead to a complete decryption of the message in the final line. Be as specific as you can. Keep in mind that we already have a function `affine_decrypt` that will produce the plaintext once the key is known. (We could use `shift_decrypt`, but remember that `affine_decrypt` is the same function as long as $m = 1$.)

<span style='color:#9c27b0'>_Please write up your answer here._</span>


*****

### Exercise 4

Let's find the key based on our guess. In our example, we suspect that the ciphertext letter "K" may correspond to the plaintext letter "e". If that is true, what shift key was used. Explain in words how you calculated it.

<span style='color:#9c27b0'>_Please write up your answer here._</span>


*****

To tell the computer to do what you explained in words for the exercise above, we need to represent the letters as numbers. ASCII to the rescue:

In [ ]:
ord("k")

In [ ]:
ord("e")

In [ ]:
ord("k") - ord("e")

### Exercise 5

Copy the subtraction code above and paste in in the code cell below, but use an uppercase "K" instead. Even though the ciphertext is "K", why would it be a bad idea to use an uppercase "K" in the code?

In [ ]:
# Add code here to subtract ASCII codes

<span style='color:#9c27b0'>_Please write up your answer here._</span>


*****

Surprisingly, we only need two lines of code to execute the basic idea:

1. Calculate the key from the guesses.
2. Decrypt the message using the key.

Here is the whole function:

In [ ]:
def shift_decrypt_freq(ciphertext, ciphertext_letter_guess, plaintext_letter_guess = "e"):
    key_guess = ord(ciphertext_letter_guess.lower()) - ord(plaintext_letter_guess.lower())
    plaintext = affine_decrypt(ciphertext, key = key_guess)
    print("key = " + str(key_guess))
    print(plaintext)

As a reminder, here is the ciphertext we're trying to decrypt:

In [ ]:
ciphertext_example1

And we plug it into the function:

In [ ]:
shift_decrypt_freq(ciphertext_example1, "K", "e")

Since "e" is the default plaintext guess, we could also simply type this:

In [ ]:
shift_decrypt_freq(ciphertext_example1, "K")

Uh oh, that doesn't look like recognizable plaintext.

### Exercise 6

Clearly, we guessed wrong. Let's assume "K" represents the second-most common English letter. Make the necessary change to the function arguments and try again. If successful, write the plaintext out using proper spacing.

In [ ]:
# Add code here to try again

<span style='color:#9c27b0'>_Please write up your answer here._</span>


### Exercise 7

Notice that the code for `shift_decrypt_freq` does not reduce anything mod 26. In fact, the second line of code will produce the following in our example:

In [ ]:
ord("K".lower()) - ord("t")

Why is this not a problem for the algorithm?

<span style='color:#9c27b0'>_Please write up your answer here._</span>


### Exercise 8

What is the correct key for this example, expressed as a number reduced modulo 26? (In other words, -9 is not the answer here, but it's related to the answer.) Check that you are correct by encrypting the plaintext using `affine_encrypt` with the correct key (and with $m = 1$) to recover the ciphertext you started with.

<span style='color:#9c27b0'>_Please write up your answer here._</span>


In [ ]:
# Add code here to encrypt the message

### Exercise 9

Let's go through all this again with a new example. Here is the ciphertext:

"BRX DUH WKH VXP WRWDO RI HYHUBWKLQJ BRX'YH HYHU VHHQ, KHDUG, HDWHQ, VPHOOHG, EHHQ WROG, IRUJRW - LW'V DOO WKHUH. HYHUBWKLQJ LQIOXHQFHV HDFK RI XV, DQG EHFDXVH RI WKDW L WUB WR PDNH VXUH WKDW PB HASHULHQFHV DUH SRVLWLYH." -PDBD DQJHORX

Use frequency analysis to decrypt the message. When you're finished, type up the plaintext using correct capitalization.

(Be careful. If you're getting errors, check the quotation marks!)

In [ ]:
# Add code here to perform a frequency analysis of the ciphertext

In [ ]:
# Add code here to apply the function shift_decrypt_freq to the ciphertext
# with your guess for the ciphertext letter and its matching plaintext letter.
# (Your guess should be based on the results of the frequency analysis above.)

*Please write up your answer here.*

### Exercise 10

Usually, we add more code to make our function robust against user error. In this case, though, we did not do that. Why not? In other words, will the function result in an error if a character other than a letter is passed to either of the latter two arguments? Try it and then explain why the function still works (even if it gives a nonsensical output).

In [ ]:
# Add code here to test the shift_decrypt_freq function with a non-letter character guess

<span style='color:#9c27b0'>_Please write up your answer here._</span>


## Frequency analysis of affine ciphers

The function for affine ciphers is going to be a little more challenging. While we can use frequency analysis to identify the most common letters, we need to figure out both the key and the value of $m$. First of all, since we have two unknowns, it won't be enough to guess one letter anymore. We'll need two. Furthermore, even with those two letters, it's not immediately obvious how to translate those guesses into a determination of the key and the value of $m$.

Here is some plaintext:

> "For instance, on the planet Earth, man had always assumed that he was more intelligent than dolphins because he had achieved so much—the wheel, New York, wars and so on—whilst all the dolphins had ever done was muck about in the water having a good time. But conversely, the dolphins had always believed that they were far more intelligent than man—for precisely the same reasons.”
― Douglas Adams, The Hitchhiker's Guide to the Galaxy

Due to the presence of both apostrophes and double quotes, we will use triple quotes to store this text:

In [ ]:
plaintext_example2 = """
For instance, on the planet Earth, man had always assumed that he was more intelligent than dolphins because he had achieved so much—the wheel, New York, wars and so on—whilst all the dolphins had ever done was muck about in the water having a good time. But conversely, the dolphins had always believed that they were far more intelligent than man—for precisely the same reasons.”
― Douglas Adams, The Hitchhiker's Guide to the Galaxy
"""

### Exercise 11

Perform a frequency analysis of the text. You should find out that the most common letters are "e" followed by "a".

In [ ]:
# Add code here to perform a frequency analysis of plaintext_example2

*****

Now let's encrypt the message, using a key of 12 and $m = 11$. (Remember that $m$ has to be relatively prime to 26.)

In [ ]:
ciphertext_example2 = affine_encrypt(plaintext_example2, key = 12, m = 11)
ciphertext_example2

We have a minor issue here. Newlines have been translated into the symbol "\n". If we leave these newlines in our text string, it will screw up our frequency analysis (inflating the number of "n"s present.) To remove these, we will re-save our plaintext without the newlines.

In [ ]:
plaintext_example2 = """For instance, on the planet Earth, man had always assumed that he was more intelligent than dolphins because he had achieved so much—the wheel, New York, wars and so on—whilst all the dolphins had ever done was muck about in the water having a good time. But conversely, the dolphins had always believed that they were far more intelligent than man—for precisely the same reasons.” ― Douglas Adams, The Hitchhiker's Guide to the Galaxy"""

In [ ]:
ciphertext_example2 = affine_encrypt(plaintext_example2, key = 12, m = 11)
ciphertext_example2

Let's look at the math for the affine cipher. Recall that we convert the plaintext to ciphertext using:

$$
c_i \equiv m p_i + k \mod 26
$$

Let's pretend that we don't know the values of $m$ and $k$. Instead, let's plug in some known plaintext letters with their corresponding ciphertext letters. For example, we can see that the plaintext letter "e" actually corresponds to "E" in the ciphertext. (This was just a lucky coincidence.) Since the letter "e" is 4 when using mod 26 (remember that "a" = 0), we can write the equation as

$$
4 \equiv m(4) + k \mod 26
$$

What about the letter "t" in the plaintext? This has been encoded as "N" in the ciphertext. "t" = 19 and "N" is 13:

$$
13 \equiv m(19) + k \mod 26
$$

This gives us a system of two linear congruences, which we know how to solve.




### Exercise 12

Solve the system of linear congruences above. In other words, find the values of $m$ and $k$. I've rewritten it below to look more like the form we've practiced:

$$
 4m + k \equiv 4 \mod 26
$$
$$
19m + k \equiv 13 \mod 26
$$

Also note that, in general, there might be multiple solutions, but in this case, there happens to be a unique solution.

You won't be able to type up all your work, so it's okay if you just report the answer you get.

<span style='color:#9c27b0'>_Please write up your answer here._</span>


### Exercise 13

Using math, explain why the plaintext "e" became the ciphertext "E".

<span style='color:#9c27b0'>_Please write up your answer here._</span>


*****

The answer you obtained by hand should not have been a surprise. After all, we can go back and see the choices that we made: the key was 12 and $m$ was 11.

Now, if we use `affine_decrypt` with the values of $k$ and $m$, we recover the plaintext:

In [ ]:
affine_decrypt(ciphertext_example2, key = 12, m = 11)

The question is, what do we do if we don't know $k$ and $m$?

This is where frequency analysis comes into play. Here are the frequencies of letters in the ciphertext:

In [ ]:
analyze_frequencies(ciphertext_example2, output_format = "percent", sort = True)

If we match up the ciphertext frequencies to the English language, we might guess that the ciphertext "E" (the most common ciphertext letter) is the plaintext letter "e" (the most common English letter) and that the ciphertext letter "M" (the second-most common ciphertext letter) is the plaintext letter "t" (the second-most common English letter). THIS TURNS OUT TO BE AN INCORRECT GUESS! But, of course, we don't know that when all we have is the frequency analysis of the ciphertext. We have no choice but to follow this lead, as it's our best guess.

### Exercise 14

Find the letter value of "M". (Not the ASCII value, but the value between 0 and 25.) Set up the system of linear congruences this implies. The first linear congruence---the one for "E" and "e"---will be the same as before, but the second one has one number that will change. In other words, the new system will look like

$$
4m + k \equiv 4 \mod 26
$$
$$
19m + k \equiv ? \mod 26
$$

But you need to fill in the question mark and then solve this system by hand. (Again, just report the answer you get for $k$ and $m$.)

*Please write up your answer here.*

### Exercise 15

If you did the exercise above correctly, you should have obtained $k = 14$ and $m = 4$. Normally, you would use those values to try to decrypt the ciphertext using the code

```
affine_decrypt(ciphertext_example2, key = 14, m = 4)
```

However, in this case, you can be sure that you have an incorrect $k$ and $m$ before even trying to decrypt the ciphertext. Why? (Hint: do you see anything wrong with the value $m = 4$?)

<span style='color:#9c27b0'>_Please write up your answer here._</span>


### Exercise 16

Now that we know we guessed incorrectly, we need another guess. Given the dominant prevalance of the "E" in the ciphertext, it seems likely that we got that one correct. It may be that our guess for "M" is wrong. (Secretly, we already know that to be the case.) So what is the third-most common letter in the English language? Perhaps the "M" corresponds to that letter?

Set up a new system of linear congruences for this new guess and solve it for $k$ and $m$. This time, it will look like the following:

$$
4m + k \equiv 4 \mod 26
$$
$$
?m + k \equiv 12 \mod 26
$$

But you need to figure out what goes in place of the question mark (the number for the third-most common letter in English). **Careful**: there will be two solutions, but only one of them makes sense. The one that works will be the right answer this time.

In the space below, list both solutions you obtained and explain why only one of the solutions makes sense.

<span style='color:#9c27b0'>_Please write up your answer here._</span>


## Can we make the computer do the yucky part?

Even though we can encrypt and decrypt affine ciphers with code, one hard part still remains: solving the system of linear congruences that results from a frequency analysis. If we happen to guess the two most frequent letters correctly, we only have to solve one system of congruences (and that's bad enough), but if we guess wrong, who knows how many other pairs of letters we may have to guess before hitting upon the correct ones.

We would like to automate this whole process, including the part about solving linear congruences. There are two ways we could proceed:

1. We could try to go through all the steps of our procedure for solving systems of linear congruences, and then try to translate everything into code. This seems...hard.
2. We could also acknowledge that computers are fast, and so maybe we can just try every possible value of $k$ and $m$ to get all possible solutions. There are only 25 possible values of $k$ and 12 possible values of $m$. That's only 300 values to test.

While option (1) is certainty possible, it's not even clear if it would be more efficient. Certainly, the code would be much messier. We'll go with option (2).

In [ ]:
def solve_affine_congruences(c1, c2, p1, p2, mod = 26):
    solution_list = []
    m_list = generate_units(mod)
    for k in range(0, mod):
        for m in m_list:
            # Check if c = m*p + k
            # or, equivalently, c - m*p - k = 0
            if (c1 - m*p1 - k) % mod == 0 and (c2 - m*p2 - k) % mod == 0 :
                solution_list.append((k, m))
    return(solution_list)

For example, the earlier system of linear congruences

$$
4 \equiv 4m + k \mod 26
$$

$$
13 \equiv 19m + k \mod 26
$$

would be entered into the function as follows:

In [ ]:
solve_affine_congruences(c1 = 4, c2 = 13, p1 = 4, p2 = 19, mod = 26)

(When you're looking at the output, you have to remember that the value of $k$ comes first, followed by the value of $m$.)

We're now ready to define our function for solving affine ciphers using frequency analysis. We'll use our helper functions `convert_letter_to number` and `convert_number_to_letter`, but don't forget that those functions only work for lowercase letters. Here is the grand finale:

In [ ]:
def affine_decrypt_freq(ciphertext, ciphertext_letter1_guess, ciphertext_letter2_guess, \
                        plaintext_letter1_guess = "e", plaintext_letter2_guess = "t"):
    # Convert letters to numbers
    p1_guess = convert_letter_to_number(plaintext_letter1_guess.lower())
    c1_guess = convert_letter_to_number(ciphertext_letter1_guess.lower())
    p2_guess = convert_letter_to_number(plaintext_letter2_guess.lower())
    c2_guess = convert_letter_to_number(ciphertext_letter2_guess.lower())
    # Solve system of linear congruences
    solution_list = solve_affine_congruences(c1_guess, c2_guess, p1_guess, p2_guess, mod = 26)
    if solution_list:
        for solution_pair in solution_list:
            print("(k, m) = " + str(solution_pair))
            print(affine_decrypt(ciphertext, key = solution_pair[0], m = solution_pair[1]))
    else:
        print("No solution")

Everything in the above function should be clear with two exceptions:

1. In the function signature, there is a backslash \ at the end of the line. All this is doing is allowing us to keep writing the code on a new line without giving us an error. The function is just easier to read if we can let the parameters spill onto another line. By the way, the default arguments for the plaintext guesses are "e" and "t" because these are the two most common letters in English text.
2. The line

```
if solution_list:
```

seems incomplete. If the solution list is what? This is simply checking that the solution list is not empty. If there are solutions, then `solution_list` will be `True` in this if statement. If the solution list is empty, `solution_list` will be `False` and we will go down to the `else` clause.

We now test the new function. Here is our earlier incorrect guess. (Remember that in the function signature, the plaintext guesses are, by default, "e" and "t".)

In [ ]:
affine_decrypt_freq(ciphertext_example2, "E", "M")

The correct guess:

In [ ]:
affine_decrypt_freq(ciphertext_example2, "E", "M", "e", "a")

### Exercise 17

Decrypt the following ciphertext using frequency analysis:

QV UKE VZA BAEV OR VQGAE, QV UKE VZA UONEV OR VQGAE, QV UKE VZA KIA OR UQEJOG, QV UKE VZA KIA OR ROOPQEZXAEE, QV UKE VZA AFOSZ OR BAPQAR, QV UKE VZA AFOSZ OR QXSNAJMPQVC, QV UKE VZA EAKEOX OR PQIZV, QV UKE VZA EAKEOX OR JKNYXAEE, QV UKE VZA EFNQXI OR ZOFA, QV UKE VZA UQXVAN OR JAEFKQN, UA ZKJ ADANCVZQXI BARONA ME, UA ZKJ XOVZQXI BARONA ME, UA UANA KPP IOQXI JQNASV VO ZAKDAX, UA UANA KPP IOQXI JQNASV VZA OVZAN UKC – QX EZONV, VZA FANQOJ UKE EO RKN PQYA VZA FNAEAXV FANQOJ, VZKV EOGA OR QVE XOQEQAEV KMVZONQVQAE QXEQEVAJ OX QVE BAQXI NASAQDAJ, RON IOOJ ON RON ADQP, QX VZA EMFANPKVQDA JAINAA OR SOGFKNQEOX OXPC.


In [ ]:
# Add code here to conduct a frequency analysis on the text

In [ ]:
# Add code here to apply the function affine_decrypt_freq to the ciphertext
# with your guesses for two ciphertext letters and their two matching plaintext letters.
# (Your guesses should be based on the results of the frequency analysis above.)

### Exercise 18

Decrypt the following ciphertext using frequency analysis:

JM JH V UVC, UVC APMMPC MEJIZ MEVM J KN, MEVI J EVWP PWPC KNIP; JM JH V UVC, UVC APMMPC CPHM J ZN MN MEVI J EVWP PWPC TINBI

(Hint: if your first few guesses don't work, keep trying!)

In [ ]:
# Add code here to conduct a frequency analysis on the text

In [ ]:
# Add code here to apply the function affine_decrypt_freq to the ciphertext
# with your guesses for two ciphertext letters and their two matching plaintext letters.
# (Your guesses should be based on the results of the frequency analysis above.)

### Exercise 19

Test to see if you can use the commands you learned here to break the code on page 99 of your coursepack. Remember, you stored it in this python module as `coursepack_encryption`. If it worked, what does that tell you about the type of cipher used in the coursepack exercise?

In [3]:
# Add code here to apply the function affine_decrypt_freq to the ciphertext
# with your guesses for two ciphertext letters and their two matching plaintext letters.
# (Your guesses should be based on the results of the frequency analysis 
## from Exercise 1b.)

## Conclusion

We have now automated the majority of our encryption and decryption tasks. The result is a collection of functions that can be used in any cryptological situation that involves either a shift or affine cipher, or one that benefits from frequency analysis.

Along the way, we've learned a little bit about how Python code works. More importantly, we've learned how to think *algorithmically*; that is, we are hopefully getting better at solving complex problems by breaking them down into small, easy-to-execute steps.